In [10]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [27]:
class MeanRevStrategy:
    def __init__(self, price_data):
        self.price_data = price_data  # DataFrame with timestamp indexed asset prices
        self.leverage = 50
        self.initial_portfolio_value = 2000
    
    def calculate_zscore(self, market):
            spread_series = self.price_data[market]
            mean = spread_series.rolling(window=300).mean()
            std = spread_series.rolling(window=300).std()
            self.price_data[f'z_score_{market}'] = (spread_series - mean) / std

    def simulate_trade(self, market, POSITION_SIZE, Z_SCORE):
        # # Calculate Z-scores for the given market
        self.calculate_zscore(market)

        # Initialize portfolio values tracking
        portfolio_value = self.initial_portfolio_value
        in_position = False
        position_type = None  # Track whether the position is long or short
        
        z_score_column = f'z_score_{market}'
        price_column = market
        
        for _, row in self.price_data.iterrows():
            current_z_score = row[z_score_column]
            current_price = row[price_column]
            trade_return = 0.0

            # Flipping positions based on Z_SCORE threshold
            if in_position:
            # Calculate return based on price change since entry
                price_increase = (current_price - entry_price) * ((POSITION_SIZE * self.leverage) / entry_price)
                if position_type == "long" and current_z_score >= Z_SCORE:
                    trade_return = price_increase - (POSITION_SIZE * self.leverage) * 0.02
                    in_position, position_type, entry_price = False, None, None
                        
            elif not in_position:
                if current_z_score <= -Z_SCORE:  # Enter long position
                    in_position, position_type, entry_price = True, "long", current_price
                


            portfolio_value += trade_return


        # Exiting the last open position at the end of the data
        if in_position:
            # Simulate exiting the position with no additional profit or loss
            # You may adjust this part based on your strategy for exiting the final open position
            in_position, position_type = False, None

        # Returning the portfolio values for further analysis
        return portfolio_value

    def run_monte_carlo_simulation(self, iterations):
        best_parameters = {}
        highest_returns = -np.inf

        for _ in range(iterations):
            WINDOW = 300
            POSITION_SIZE = 1
            Z_SCORE = np.random.uniform(2.5, 3.5)
            total_returns = self.trade_all_markets(WINDOW, POSITION_SIZE, Z_SCORE)
            # Check if this simulation yielded higher returns than previous best
            if total_returns > highest_returns:
                highest_returns = total_returns
                best_parameters = {
                    'returns': highest_returns,
                    'window': WINDOW,
                    'position_size': POSITION_SIZE,
                    'z_score': Z_SCORE
                }

        # Save the best parameters to a JSON file
        with open('best_parameters.json', 'w') as f:
            json.dump(best_parameters, f, indent=4)

        return best_parameters

    def trade_all_markets(self, WINDOW, POSITION_SIZE, Z_SCORE):
        
        total_returns = 0.0
        for market in self.price_data:
            if market == 'time' or market.startswith('z_score_'):
                continue

            portfolio_value = self.simulate_trade(market, POSITION_SIZE, Z_SCORE)
            returns = portfolio_value - self.initial_portfolio_value
            total_returns += returns
           

        return total_returns

In [32]:
data_15m = pd.read_csv('data_test_15m.csv')
data_15m = data_15m.tail(2000)


scalping_15m = MeanRevStrategy(data_15m)
for market in scalping_15m.price_data:
    # Skip 'time' or any non-market column if present
    if market == 'time':
        continue
    scalping_15m.calculate_zscore(market)
scalping_15m.run_monte_carlo_simulation(100)

{'returns': 64.29350903945328,
 'window': 300,
 'position_size': 1,
 'z_score': 3.0511790550534275}

In [33]:
data_1H = pd.read_csv('data_test_1H.csv')
data_1H = data_1H.tail(2000)


scalping_1H = MeanRevStrategy(data_1H)
for market in scalping_1H.price_data:
    # Skip 'time' or any non-market column if present
    if market == 'time':
        continue
    scalping_1H.calculate_zscore(market)
scalping_1H.run_monte_carlo_simulation(100)

{'returns': 29.707762670467673,
 'window': 300,
 'position_size': 1,
 'z_score': 2.9940537039221518}